### Objective: Predict House Sales Prices, target 80% and above prediction score
#### Data: Kaggle - House Sales in King County, USA

In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import math

In [2]:
# to check current directory

import os
os.getcwd()

# or also can use %pwd to view current directory in Jupyter

'C:\\Users\\Kerry\\Desktop\\DataKhng\\Multiple Linear Regression\\Example 1'

In [3]:
# to check files in current directory

#os.listdir(os.getcwd())

### 1. Data Inspections

In [4]:
# read data into dataframe

data = pd.read_csv('kc_house_data.csv')

In [5]:
# display data

data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


#### This is a cross-sectional data
#### Data Description obtained from: https://rstudio-pubs-static.s3.amazonaws.com/155304_cc51f448116744069664b35e7762999f.html
id - Unique ID for each home sold
<br />
date - Date of the home sale
<br />
price - Price of each home sold
<br />
bedrooms - Number of bedrooms
<br />
bathrooms - Number of bathrooms, where .5 accounts for a room with a toilet but no shower
<br />
sqft_living - Square footage of the apartments interior living space
<br />
sqft_lot - Square footage of the land space
<br />
floors - Number of floors
<br />
waterfront - A dummy variable for whether the apartment was overlooking the waterfront or not
<br />
view - An index from 0 to 4 of how good the view of the property was
<br />
condition - An index from 1 to 5 on the condition of the apartment
<br />
grade - An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design
<br />
sqft_above - The square footage of the interior housing space that is above ground level
<br />
sqft_basement - The square footage of the interior housing space that is below ground level
<br />
yr_built - The year the house was initially built
<br />
yr_renovated - The year of the house’s last renovation
<br />
zipcode - What zipcode area the house is in
<br />
lat - Lattitude
<br />
long - Longitude
<br />
sqft_living15 - The square footage of interior housing living space for the nearest 15 neighbors
<br />
sqft_lot15 - The square footage of the land lots of the nearest 15 neighbors
<br />
#### NOTE: sqft_living15 anf sqft_lot15 were defined differently by different analysts

In [6]:
# check the number of data points(rows) and columns in the dataset
# print(data.shape)

# to get summary of the dataframe
# data.info()

# check the number of data points(rows)in the dataset

print(len(data))

# check the data types

data.dtypes

21613


id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

#### date is the python object, and it is the timestamp we will ignore
#### we will drop id and date

In [7]:
# Calling sum() of the DataFrame returned by isnull() will give a series containing data about count of NaN in each column
# data.isnull().sum()

# check any number of columns with NaN

print(data.isnull().any().sum(),'/',len(data.columns))

# check any number of data points(rows) with NaN

print(data.isnull().any(axis=1).sum(),'/',len(data))

0 / 21
0 / 21613


#### This dataset got no NaN values

In [8]:
# check correlation between features(independent variables) and target(dependent variable = price)

features = data.iloc[:,3:].columns.tolist()
target = data.iloc[:,2].name

In [9]:
# using Pearson's correlation method

from scipy.stats import pearsonr

correlations = {}
for f in  features: 
    dataset = data[[f,target]]
    x1 = dataset[f].values
    x2 = dataset[target].values
    key = f + 'vs' + target
    correlations[key] = pearsonr(x1,x2)[0]

In [10]:
data_correlations = pd.DataFrame(correlations, index=['Value']).T
data_correlations.loc[data_correlations['Value'].abs().sort_values(ascending=False).index]

,Value
sqft_livingvsprice,0.702035
gradevsprice,0.667434
sqft_abovevsprice,0.605567
sqft_living15vsprice,0.585379
bathroomsvsprice,0.525138
viewvsprice,0.397293
sqft_basementvsprice,0.323816
bedroomsvsprice,0.308350
latvsprice,0.307003
waterfrontvsprice,0.266369


#### Top 5 features (sqft_living, grade, sqft_above, sqft_living15, and bathrooms) are the most correlated features with price.

### 2. Model Testing

#### We hardly just rely on correlations to decide which variable is suitable so we test it one by one.
#### Here, we look at coefficients and consider theoretical concepts for variables selection.
#### Statistically, we can select confidence interval of lets say 95%, conduct t-test, look at p-value (lower than apha = 0.05 stands significant), look at R square etc. to determine whether to drop or remain a variable in the model. 

In [11]:
# predict the house sale prices

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

reg = LinearRegression()
new_data = data[['sqft_living','grade', 'sqft_above', 'sqft_living15','bathrooms','view','sqft_basement', 'bedrooms', 'lat', 'waterfront', 'floors', 'yr_renovated', 'sqft_lot', 'sqft_lot15', 'yr_built', 'zipcode', 'condition', 'long']]

In [12]:
X = new_data.values
y = data.price.values

In [13]:
# Splitting the dataset into the Training set and Test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [14]:
# Fitting Linear Regression to the Training set

reg.fit(X_train, y_train)

# Predicting the Test set results

print(reg.predict(X_test))

[ 304621.40153677  752308.8738118   282175.86784607 ... 1210542.73252309
  268547.1163622   724910.16577461]


In [15]:
print('coefficient', reg.coef_)

coefficient [ 1.10988123e+02  9.39643231e+04  7.46626722e+01  2.31158765e+01
  4.51474359e+04  5.47412793e+04  3.63254502e+01 -3.69320031e+04
  6.05455886e+05  5.83457388e+05  4.04941398e+03  1.91193738e+01
  1.58457365e-01 -4.22352207e-01 -2.68948244e+03 -5.97804482e+02
  2.58624942e+04 -2.25264183e+05]


In [16]:
reg.score(X_test, y_test)

0.7120725762091098

In [17]:
# get the mean squared error using scikit-learn’s mean_squared_error method and 
# comparing the prediction for the test data set (data not used for training) with the ground truth for the data test set

from sklearn.metrics import mean_squared_error

y_predict = reg.predict(X_test)
regression_model_rmse = float(format(math.sqrt(mean_squared_error(y_test, y_predict)),'.3f'))
regression_model_rmse

187817.686

In [18]:
# Another method to calculate the Root Mean Squared Error

print("RMSE: %.2f"
% math.sqrt(np.mean((reg.predict(X_test) - y_test) ** 2)))

RMSE: 187817.69


#### By looking at coefficient results: features which we can try to drop are sqft_lot15, sqft_lot.

In [19]:
regr = LinearRegression()
new_data1 = data[['sqft_living','grade', 'sqft_above', 'sqft_living15','bathrooms','view','sqft_basement', 'bedrooms', 'lat', 'waterfront', 'floors', 'yr_renovated', 'yr_built', 'zipcode', 'condition', 'long']]

In [20]:
X1 = new_data1.values
y = data.price.values

In [21]:
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size = 0.2)

In [22]:
regr.fit(X1_train, y_train)

print(regr.predict(X1_test))

[335670.26037281 817642.86757261 273773.09833566 ... 642839.76130122
 608356.42043998 397909.5214478 ]


In [23]:
print('coefficient', regr.coef_)

coefficient [ 1.16163561e+02  9.60413909e+04  7.27648030e+01  1.28865027e+01
  4.26363135e+04  5.27739868e+04  4.33987578e+01 -3.99607966e+04
  6.00577084e+05  5.84963159e+05  7.47754108e+03  1.78379588e+01
 -2.65336257e+03 -6.10062820e+02  2.50608887e+04 -2.22170337e+05]


In [24]:
print(regr.score(X1_test, y_test))

0.7026359000867606


In [25]:
print("RMSE: %.2f"
% math.sqrt(np.mean((regr.predict(X1_test) - y_test) ** 2)))

RMSE: 194799.25


#### Lets try to drop zipcode and yr_renovated by taking coefficient and correlation results into consideration. 

In [26]:
regre = LinearRegression()
new_data2 = data[['sqft_living','grade', 'sqft_above', 'sqft_living15','bathrooms','view','sqft_basement', 'bedrooms', 'lat', 'waterfront', 'floors', 'yr_built', 'condition', 'long']]

In [27]:
X2 = new_data2.values
y = data.price.values

In [28]:
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, test_size = 0.2)

In [29]:
regre.fit(X2_train, y_train)

print(regre.predict(X2_test))

[ 617673.80196141  605461.73728961  563763.52742425 ... 1029289.32485318
  194068.8343555   497561.51997047]


In [30]:
print('coefficient', regre.coef_)

coefficient [ 1.07257801e+02  9.76601081e+04  6.94267947e+01  2.84335741e+01
  4.80695840e+04  4.88917850e+04  3.78310065e+01 -3.78383033e+04
  5.55152600e+05  6.29961367e+05  3.34548397e+03 -2.66799643e+03
  2.89188332e+04 -1.09418567e+05]


In [31]:
print(regre.score(X2_test, y_test))

0.6909062587512115


In [32]:
print("RMSE: %.2f"
% math.sqrt(np.mean((regre.predict(X2_test) - y_test) ** 2)))

RMSE: 202219.41


#### Lets drop long and lat by considering three aspects (coefficient results, correlation results, and theoretical concepts).

In [33]:
regres = LinearRegression()
new_data3 = data[['sqft_living','grade', 'sqft_above', 'sqft_living15','bathrooms','view','sqft_basement', 'bedrooms', 'waterfront', 'floors', 'yr_built', 'condition']]

In [34]:
X3 = new_data3.values
y = data.price.values

In [35]:
X3_train, X3_test, y_train, y_test = train_test_split(X3, y, test_size = 0.2)

In [36]:
regres.fit(X3_train, y_train)

print(regres.predict(X3_test))

[596644.4152111 498852.4152111 327844.4152111 ... 424612.4152111
 757412.4152111 420516.4152111]


In [37]:
print('coefficient', regres.coef_)

coefficient [-1.55185671e+15  1.19467675e+05  1.55185671e+15  2.36812969e+01
  4.79804452e+04  4.31180161e+04  1.55185671e+15 -3.74300885e+04
  5.67192406e+05  3.04399401e+04 -3.66918700e+03  1.81542765e+04]


In [38]:
print(regres.score(X3_test, y_test))

0.6587867226344539


In [39]:
print("RMSE: %.2f"
% math.sqrt(np.mean((regres.predict(X3_test) - y_test) ** 2)))

RMSE: 212791.74


#### Overall, 60-70% of the variability in y can be explained using X (i.e. prediction score/accuracy measure of this model is around 60-70% which is not really optimal). 

#### Model 1: R squared = 0.7120725762091098; RMSE = 187817.69
#### Model 2: R squared = 0.7026359000867606; RMSE = 194799.25
#### Model 3: R squared = 0.6909062587512115; RMSE = 202219.41
#### Model 4: R squared = 0.6587867226344539; RMSE = 212791.74

#### Model 1 displays better result among all other models with highest prediction score, and the average difference between the predicted and the actual price for test data set is the lowest (i.e. 187817.69 dollars).

### We need to use other models to test and train again in order to achieve higher prediction score. 